In [ ]:
import sys
import os
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pathlib import Path
import tensorflow as tf
import pandas as pd
import numpy as np

from utilities import create_model
from utilities import TestConfiguration
from utilities import TestDataset
config = TestConfiguration()

IMAGE_SET_NAME = 'standley_4058_test' # 'IAM_Words_test'
METADATA_FILENAME = 'words_metadata.csv'

## Load the test set locally

In [ ]:
# OPTIONAL: Download test set from Google Cloud Storage
IMAGE_BUCKET = 'fmnh_datasets'

storage_path = f'gs://{IMAGE_BUCKET}/{IMAGE_SET_NAME}/'
!gsutil -m cp -r $storage_path ../images

In [ ]:
local_image_location = f'../images/{IMAGE_SET_NAME}'
data_dir = Path(local_image_location)

test_dataset = TestDataset()
test_dataset.create_dataset(32, data_dir, METADATA_FILENAME)

## Model loading

In [ ]:
# Load model from Google Cloud Storage
MODEL_BUCKET = 'fmnh_models'
MODEL_NAME = '13-iam_train-run_55-standley_train-100e_01lr_fine_tuned_100e'
if not os.path.exists('models'):
    os.makedirs('models')

model_uri = f'gs://{MODEL_BUCKET}/{MODEL_NAME}'
!gsutil -m cp -r $model_uri ./models/
prediction_model_filename = Path(f'./models/{MODEL_NAME}')
prediction_model = tf.keras.models.load_model(prediction_model_filename)

In [ ]:
# OR, load model from local filesystem
MODEL_NAME = 'fine_tuned-prediction'
model_location = Path(f'../transfer_learning/{MODEL_NAME}')
prediction_model = tf.keras.models.load_model(model_location)

In [ ]:
# OR, use already-downloaded model
MODEL_NAME = 'iam_train-run_55'
model_location = Path(f'./models/{MODEL_NAME}')
prediction_model = tf.keras.models.load_model(model_location)

In [ ]:
opt = tf.keras.optimizers.Adam()
prediction_model.compile(optimizer=opt)

## Prediction generation

In [ ]:
prediction_results = pd.DataFrame(columns=['label', 'prediction'])
for batch in test_dataset.test_dataset:
    images = batch['image']
    labels = batch['label']
    preds = prediction_model.predict(images)
    pred_texts = test_dataset.decode_batch_predictions(preds)
    pred_texts = [t.replace('[UNK]', '').replace(' ', '') for t in pred_texts]
    orig_texts = []
    for label in labels:
        label = tf.strings.reduce_join(test_dataset.num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)
    orig_texts = [t.replace('[UNK]', '').replace(' ', '') for t in orig_texts]
    new_results = pd.DataFrame(zip(orig_texts, pred_texts), columns=['label', 'prediction'])
    prediction_results = prediction_results.append(new_results, ignore_index=True)
print(prediction_results)

if not os.path.exists('predictions'):
    os.makedirs('predictions')
prediction_results.to_csv(Path('predictions', f'{MODEL_NAME}-predictions.csv'))